# Youtube Comment Crawling

### Youtube 댓글을 크롤링하는 목적
- 예시
    - 브이로그의 댓글이 악플이 달렸는지 선플이 달렸는지 확인을 하고 싶다. -> 브이로그의 주소를 받아서 쓰면 된다
    - 축구경기를 할 때 사람들의 반을을 보고 싶다. -> 유튜브 검색창에 축구를 치고 영상에 들어가서 댓글을 크롤링하면 된다.

- 목적
    - 상황: 대형 유튜버인 내가 내 유튜브 영상에 있는 댓글들에서 악플을 찾고 싶다

- 과제 목록
    1. scroll 횟수 = 5번
    2. 댓글 수집 시작
        1. 대댓글 ❌
        2. ID / 댓글 / 좋아요 Data 크롤링
        3. Data에 video_id열 추가 (`v=video_id`)

### Youtube 크롤링을 하기 전 결정해야 할 일

- Selenium으로 만들기

### URL 파악하기
- Youtube의 도메인은 "https://www.youtube.com/"
- 내가 크롤링하고자 하는 영상의 주소는 "https://www.youtube.com/watch?v=SsJg6x-inFw"
- 키
    - `watch?v=`
        - 뒤에 영상의 고유주소

In [265]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [284]:
# 크롬창 열기
options = Options()
driver = webdriver.Chrome(options=options)
driver.maximize_window()
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
video_id = "SsJg6x-inFw"
# 유튜브 주소 들어가기
driver.get("https://www.youtube.com/watch?v=" + video_id)



In [288]:
# 스크롤 5번하기
n_scroll = 5
ActionChains(driver).send_keys("k").perform()
last_height = driver.execute_script("return document.documentElement.scrollHeight")

# for i in range(5):
    
#     driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    
    
#     new_height = driver.execute_script("return document.documentElement.scrollHeight")
#     if new_height == last_height:
#         break
#     last_height = new_height
    

# time.sleep(2)

# 페이지를 5번 스크롤
# for _ in range(n_scroll):
#     # JavaScript를 사용하여 페이지 스크롤
#     driver.execute_script("window.scrollBy(0, window.innerHeight);")
#     # 각 스크롤 후 약간의 지연을 추가하여 페이지가 로드될 시간을 줍니다.
#     time.sleep(1)
#     wait = WebDriverWait(driver, 10)
#     wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

container=driver.find_element(By.TAG_NAME, "html")
before_top = driver.execute_script("return arguments[0].scrollTop", container)

for i in range(5):
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", container)
    after_top = driver.execute_script("return arguments[0].scrollTop", container)

    # if int(before_top) == int(after_top):
    #     break
        
    before_top = after_top 

In [167]:
# 유튜브 프리미엄 창 없애기
try:
    driver.find_element(By.CSS_SELECTOR, "#dismiss-button > a").click()
except:
    pass

In [291]:
# temp = driver.find_element(By.ID, "comments").find_element(By.ID, "contents").find_elements(By.CLASS_NAME, "style-scope.ytd-item-section-renderer")
temp = driver.find_elements(By.ID, "context-text")
print(temp)

# 아이디 받아오기
# y_id = temp[1].find_element(By.ID,  'author-text').text
# print(y_id)

[]


In [295]:
temps = driver.find_elements(By.CSS_SELECTOR, '#content-text')
for i in range(5):
    print(temps[i].text)


한화의 김경문, 롯데의 김태형.... 오랫동안 꼴찌 였던 팀들이 서서히 자리가 잡혀가고 있음. 이래서 감독이 중요한 거구나...
또또또 설레발친다. 지나가던 한화팬입니다.
이거 한화 계속 본 사람들은 공감할거임.
문현빈이 롯데전 이후로 병살타로 맛이 가있었고, 하필 1루에 주자 있는데 대타 문현빈 보고 이게 뭔가 했음. 그런데 이걸 스퀴즈 번트로 역전도 해, 클러치상황만 오면 병살타를 치던 문현빈이 짜릿하게 웃고있어. 그저 대 경 문.
코칭스텝 , 구단관계자 , 선수여러분 양심이 있다면 25년을 기다리고 있는 김승연 회장님과 한화팬들을 위해 1999년을 재현 합시다.
아니...한화가 작전야구를!?!


In [195]:
# 댓글 받아오기
y_comment = temp[0].find_element(By.ID, 'content-text').text
print(y_comment)

한화의 김경문, 롯데의 김태형.... 오랫동안 꼴찌 였던 팀들이 서서히 자리가 잡혀가고 있음. 이래서 감독이 중요한 거구나...


In [196]:
# 좋아요 받아오기
y_like = temp[0].find_element(By.ID, 'vote-count-middle').text
print(y_like)

456


In [222]:
print(len(y_id))

16


In [275]:
data_dict = {'ID':[],'Comments':[],'Likes':[]}


for i in range(len(temp)):
    y_id = temp[i].find_element(By.ID,  'author-text').text
    y_comment = temp[i].find_element(By.ID, 'content-text').text
    y_like = temp[i].find_element(By.ID, 'vote-count-middle').text
    
    data_dict["ID"].append(y_id)
    data_dict["Comments"].append(y_comment)
    data_dict["Likes"].append(y_like)

pd.DataFrame(data_dict)

StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=125.0.6422.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF65C3B1F52+60322]
	(No symbol) [0x00007FF65C32CEC9]
	(No symbol) [0x00007FF65C1E7EBA]
	(No symbol) [0x00007FF65C1EDCEE]
	(No symbol) [0x00007FF65C1F0641]
	(No symbol) [0x00007FF65C1F06E0]
	(No symbol) [0x00007FF65C23733B]
	(No symbol) [0x00007FF65C23773C]
	(No symbol) [0x00007FF65C22AEEC]
	(No symbol) [0x00007FF65C25C25F]
	(No symbol) [0x00007FF65C22ADB6]
	(No symbol) [0x00007FF65C25C430]
	(No symbol) [0x00007FF65C27BC80]
	(No symbol) [0x00007FF65C25BFC3]
	(No symbol) [0x00007FF65C229617]
	(No symbol) [0x00007FF65C22A211]
	GetHandleVerifier [0x00007FF65C6C94AD+3301629]
	GetHandleVerifier [0x00007FF65C7136D3+3605283]
	GetHandleVerifier [0x00007FF65